# Task 3: Employee Salary Prediction using Multiple Regression

## Objective
Organisations rely on predictive analytics to determine fair and competitive salaries based on employee attributes. Machine learning models help identify salary trends and ensure data-driven compensation decisions.

In this notebook, we will:
1. Load and explore the employee salary dataset
2. Preprocess data including encoding categorical variables
3. Build a predictive model using multiple regression techniques
4. Analyze coefficients and their significance
5. Achieve a good R² score
6. Answer the assignment questions about feature influence and practical implications

### Dataset Features:
- **Experience**: Years of experience
- **Education**: Education level
- **Age**: Employee age
- **Location**: City/Location
- **Role**: Job role
- **Salary**: Target variable

---
## 1. Import Required Libraries

In [ ]:
# Data manipulation and analysis
import pandas as pd
import numpy as np

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns

# Machine Learning
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

# Statistical analysis
import statsmodels.api as sm
from scipy import stats

# Settings
import warnings
warnings.filterwarnings('ignore')
plt.style.use('seaborn-v0_8-whitegrid')
pd.set_option('display.max_columns', None)

print("All libraries imported successfully!")

---
## 2. Load and Explore the Dataset

In [ ]:
# Load the employee salary dataset
df = pd.read_csv('../Task_3_Assets/employee_salary_dataset.csv')

# Display basic information
print("="*60)
print("DATASET OVERVIEW")
print("="*60)
print(f"\nShape of dataset: {df.shape[0]} rows, {df.shape[1]} columns")
print(f"\nColumn names: {df.columns.tolist()}")
print("\n" + "="*60)
print("FIRST 10 ROWS")
print("="*60)
df.head(10)

In [ ]:
# Check data types and missing values
print("DATA TYPES AND NON-NULL COUNTS")
print("="*60)
print(df.info())

print("\n" + "="*60)
print("MISSING VALUES")
print("="*60)
missing = df.isnull().sum()
print(missing[missing > 0] if missing.sum() > 0 else "No missing values found!")

print("\n" + "="*60)
print("STATISTICAL SUMMARY")
print("="*60)
df.describe()

In [ ]:
# Explore categorical variables
print("CATEGORICAL VARIABLE DISTRIBUTIONS")
print("="*60)

categorical_cols = ['Education', 'Location', 'Role']
for col in categorical_cols:
    print(f"\n{col}:")
    print(df[col].value_counts())

---
## 3. Exploratory Data Analysis (EDA)

In [ ]:
# Distribution of target variable (Salary)
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Histogram
axes[0].hist(df['Salary'], bins=30, color='steelblue', edgecolor='black', alpha=0.7)
axes[0].set_title('Distribution of Salary', fontsize=14, fontweight='bold')
axes[0].set_xlabel('Salary ($)')
axes[0].set_ylabel('Frequency')
axes[0].axvline(df['Salary'].mean(), color='red', linestyle='--', label=f'Mean: ${df["Salary"].mean():,.0f}')
axes[0].axvline(df['Salary'].median(), color='green', linestyle='--', label=f'Median: ${df["Salary"].median():,.0f}')
axes[0].legend()

# Box plot
axes[1].boxplot(df['Salary'], vert=True)
axes[1].set_title('Box Plot of Salary', fontsize=14, fontweight='bold')
axes[1].set_ylabel('Salary ($)')

plt.tight_layout()
plt.show()

print(f"\nSalary Statistics:")
print(f"  Mean: ${df['Salary'].mean():,.2f}")
print(f"  Median: ${df['Salary'].median():,.2f}")
print(f"  Std Dev: ${df['Salary'].std():,.2f}")

In [ ]:
# Salary by categorical variables
fig, axes = plt.subplots(1, 3, figsize=(16, 5))

# Salary by Education
df.boxplot(column='Salary', by='Education', ax=axes[0])
axes[0].set_title('Salary by Education Level', fontsize=12, fontweight='bold')
axes[0].set_xlabel('Education')
axes[0].set_ylabel('Salary ($)')

# Salary by Location  
df.boxplot(column='Salary', by='Location', ax=axes[1])
axes[1].set_title('Salary by Location', fontsize=12, fontweight='bold')
axes[1].set_xlabel('Location')
axes[1].set_ylabel('Salary ($)')
axes[1].tick_params(axis='x', rotation=45)

# Salary by Role
df.boxplot(column='Salary', by='Role', ax=axes[2])
axes[2].set_title('Salary by Role', fontsize=12, fontweight='bold')
axes[2].set_xlabel('Role')
axes[2].set_ylabel('Salary ($)')
axes[2].tick_params(axis='x', rotation=45)

plt.suptitle('')  # Remove automatic title
plt.tight_layout()
plt.show()

In [ ]:
# Scatter plots for numerical variables vs Salary
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Experience vs Salary
axes[0].scatter(df['Experience'], df['Salary'], alpha=0.5, color='blue', edgecolor='black', linewidth=0.3)
z = np.polyfit(df['Experience'], df['Salary'], 1)
p = np.poly1d(z)
axes[0].plot(df['Experience'].sort_values(), p(df['Experience'].sort_values()), color='red', linewidth=2, linestyle='--')
corr_exp = df['Experience'].corr(df['Salary'])
axes[0].set_title(f'Experience vs Salary (r = {corr_exp:.3f})', fontsize=12, fontweight='bold')
axes[0].set_xlabel('Years of Experience')
axes[0].set_ylabel('Salary ($)')

# Age vs Salary
axes[1].scatter(df['Age'], df['Salary'], alpha=0.5, color='green', edgecolor='black', linewidth=0.3)
z = np.polyfit(df['Age'], df['Salary'], 1)
p = np.poly1d(z)
axes[1].plot(df['Age'].sort_values(), p(df['Age'].sort_values()), color='red', linewidth=2, linestyle='--')
corr_age = df['Age'].corr(df['Salary'])
axes[1].set_title(f'Age vs Salary (r = {corr_age:.3f})', fontsize=12, fontweight='bold')
axes[1].set_xlabel('Age')
axes[1].set_ylabel('Salary ($)')

plt.tight_layout()
plt.show()

---
## 4. Data Preprocessing

In [ ]:
# Create a copy for preprocessing
df_processed = df.copy()

# One-hot encode categorical variables
df_encoded = pd.get_dummies(df_processed, columns=['Education', 'Location', 'Role'], drop_first=True)

print("Shape after encoding:", df_encoded.shape)
print("\nEncoded columns:")
print(df_encoded.columns.tolist())

In [ ]:
# Prepare features and target
X = df_encoded.drop('Salary', axis=1)
y = df_encoded['Salary']

# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print(f"Training set: {X_train.shape[0]} samples")
print(f"Testing set: {X_test.shape[0]} samples")
print(f"\nFeatures: {X.columns.tolist()}")

---
## 5. Building the Multiple Regression Model

In [ ]:
# Train Multiple Linear Regression model
model = LinearRegression()
model.fit(X_train, y_train)

# Make predictions
y_train_pred = model.predict(X_train)
y_test_pred = model.predict(X_test)

print("Multiple Linear Regression Model Training Complete!")
print("="*60)

In [ ]:
# Analyze model coefficients
coefficients = pd.DataFrame({
    'Feature': X.columns,
    'Coefficient': model.coef_
}).sort_values('Coefficient', ascending=False, key=abs)

print("LINEAR REGRESSION MODEL COEFFICIENTS")
print("="*80)
print(f"\nIntercept (B0): ${model.intercept_:,.2f}")
print("\nFeature Coefficients (sorted by absolute value):")
print("-"*60)
print(coefficients.to_string(index=False))

# Visualize coefficients
plt.figure(figsize=(12, 8))
colors = ['green' if c > 0 else 'red' for c in coefficients['Coefficient']]
bars = plt.barh(range(len(coefficients)), coefficients['Coefficient'], color=colors, edgecolor='black')
plt.yticks(range(len(coefficients)), coefficients['Feature'])
plt.axvline(x=0, color='black', linewidth=0.8)
plt.xlabel('Coefficient Value ($)', fontsize=12)
plt.ylabel('Feature', fontsize=12)
plt.title('Multiple Regression Coefficients\n(Impact on Salary)', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

---
## 6. Model Evaluation

In [ ]:
# Model Evaluation
print("MODEL EVALUATION METRICS")
print("="*60)

# Training metrics
r2_train = r2_score(y_train, y_train_pred)
rmse_train = np.sqrt(mean_squared_error(y_train, y_train_pred))
mae_train = mean_absolute_error(y_train, y_train_pred)

# Testing metrics
r2_test = r2_score(y_test, y_test_pred)
rmse_test = np.sqrt(mean_squared_error(y_test, y_test_pred))
mae_test = mean_absolute_error(y_test, y_test_pred)

print("\n--- Training Set ---")
print(f"R-squared: {r2_train:.4f}")
print(f"RMSE: ${rmse_train:,.2f}")
print(f"MAE: ${mae_train:,.2f}")

print("\n--- Testing Set ---")
print(f"R-squared: {r2_test:.4f}")
print(f"RMSE: ${rmse_test:,.2f}")
print(f"MAE: ${mae_test:,.2f}")

print("\n" + "="*60)
print("INTERPRETATION")
print("="*60)
print(f"\n- R-squared = {r2_test:.4f} means the model explains {r2_test*100:.2f}% of the variance in salary.")
print(f"- On average, predictions are off by approximately ${mae_test:,.2f}.")

In [ ]:
# Actual vs Predicted Plot
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Training set
axes[0].scatter(y_train, y_train_pred, alpha=0.5, color='blue', edgecolor='black', linewidth=0.3)
axes[0].plot([y_train.min(), y_train.max()], [y_train.min(), y_train.max()], 'r--', linewidth=2)
axes[0].set_xlabel('Actual Salary ($)', fontsize=11)
axes[0].set_ylabel('Predicted Salary ($)', fontsize=11)
axes[0].set_title(f'Training: Actual vs Predicted (R-sq = {r2_train:.4f})', fontsize=12, fontweight='bold')

# Testing set
axes[1].scatter(y_test, y_test_pred, alpha=0.5, color='green', edgecolor='black', linewidth=0.3)
axes[1].plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--', linewidth=2)
axes[1].set_xlabel('Actual Salary ($)', fontsize=11)
axes[1].set_ylabel('Predicted Salary ($)', fontsize=11)
axes[1].set_title(f'Testing: Actual vs Predicted (R-sq = {r2_test:.4f})', fontsize=12, fontweight='bold')

plt.tight_layout()
plt.show()

---
## 7. Statistical Analysis (OLS)

In [ ]:
# OLS Statistical Analysis
X_with_const = sm.add_constant(X)
ols_model = sm.OLS(y, X_with_const).fit()

print(ols_model.summary())

In [ ]:
# Detailed coefficient analysis with p-values
print("\n" + "="*80)
print("DETAILED COEFFICIENT ANALYSIS WITH P-VALUES")
print("="*80)

results_df = pd.DataFrame({
    'Feature': ols_model.params.index,
    'Coefficient': ols_model.params.values,
    'Std Error': ols_model.bse.values,
    't-value': ols_model.tvalues.values,
    'p-value': ols_model.pvalues.values,
    'Significant': ['Yes' if p < 0.05 else 'No' for p in ols_model.pvalues.values]
}).sort_values('p-value')

print("\n")
print(results_df.to_string(index=False))

print("\n" + "-"*80)
print("\nStatistically Significant Features (p < 0.05):")
significant = results_df[results_df['Significant'] == 'Yes']
for _, row in significant.iterrows():
    if row['Feature'] != 'const':
        print(f"   - {row['Feature']}: coef={row['Coefficient']:.2f}, p={row['p-value']:.4f}")

---
## 8. Answers to Assignment Questions

### Question 1: Feature Influence - Coefficient Interpretation

In [ ]:
# Question 1 Analysis: Feature Influence
print("="*80)
print("QUESTION 1: Feature Influence on Salary")
print("="*80)

# Get coefficients excluding intercept
coef_analysis = results_df[results_df['Feature'] != 'const'].copy()

# Positive coefficients (increase salary)
positive_coefs = coef_analysis[coef_analysis['Coefficient'] > 0].sort_values('Coefficient', ascending=False)
print("\n1. POSITIVE COEFFICIENTS (Increase Salary):")
print("-"*60)
for _, row in positive_coefs.head(5).iterrows():
    print(f"   {row['Feature']:30s}: +${row['Coefficient']:,.2f}")

# Negative coefficients (decrease salary)
negative_coefs = coef_analysis[coef_analysis['Coefficient'] < 0].sort_values('Coefficient')
print("\n2. NEGATIVE COEFFICIENTS (Decrease Salary):")
print("-"*60)
for _, row in negative_coefs.head(5).iterrows():
    print(f"   {row['Feature']:30s}: ${row['Coefficient']:,.2f}")

# Strongest influence
strongest = coef_analysis.iloc[coef_analysis['Coefficient'].abs().argmax()]
print(f"\n3. STRONGEST INFLUENCE ON SALARY:")
print("-"*60)
print(f"   Feature: {strongest['Feature']}")
print(f"   Coefficient: ${strongest['Coefficient']:,.2f}")
print(f"   p-value: {strongest['p-value']:.4f}")

#### Answer to Question 1: Coefficient Interpretation

**What do positive and negative coefficients indicate?**

1. **Positive Coefficient**: A positive coefficient means that as the feature value increases (or when a categorical feature is present), the predicted salary **increases** by the coefficient amount, holding all other variables constant.

2. **Negative Coefficient**: A negative coefficient means that as the feature value increases (or when a categorical feature is present), the predicted salary **decreases** by the coefficient amount, holding all other variables constant.

**Key Findings:**

- **Experience** typically has a strong positive coefficient, indicating that more years of experience leads to higher salary
- **Education levels** (like PhD, Master's) often have positive coefficients compared to the baseline education level
- **Location** variables show how different cities compare to the baseline location
- **Role** variables show salary differences across job roles

**Strongest Influence:**
The feature with the largest absolute coefficient has the **strongest influence** on salary prediction. This is typically Experience or certain high-level education/role categories.

### Question 2: Practical Implications

In [ ]:
# Question 2 Analysis: Practical Implications
print("="*80)
print("QUESTION 2: Practical Implications Analysis")
print("="*80)

# Analyze Experience coefficient
print("\n1. EXPERIENCE COEFFICIENT ANALYSIS:")
print("-"*60)
if 'Experience' in results_df['Feature'].values:
    exp_row = results_df[results_df['Feature'] == 'Experience'].iloc[0]
    print(f"   Coefficient: ${exp_row['Coefficient']:,.2f}")
    print(f"   p-value: {exp_row['p-value']:.4f}")
    print(f"   Significant: {exp_row['Significant']}")

# Analyze Location coefficients
print("\n2. LOCATION COEFFICIENT ANALYSIS:")
print("-"*60)
location_coefs = results_df[results_df['Feature'].str.contains('Location', case=False)]
for _, row in location_coefs.iterrows():
    print(f"   {row['Feature']:25s}: ${row['Coefficient']:,.2f} (p={row['p-value']:.4f})")

# Analyze Education coefficients
print("\n3. EDUCATION COEFFICIENT ANALYSIS:")
print("-"*60)
education_coefs = results_df[results_df['Feature'].str.contains('Education', case=False)]
for _, row in education_coefs.iterrows():
    print(f"   {row['Feature']:25s}: ${row['Coefficient']:,.2f} (p={row['p-value']:.4f})")

#### Answer to Question 2: Practical Implications

**1. If the coefficient for Experience is positive, what does it imply?**

A positive coefficient for Experience implies that salary **increases** with more years of experience. This makes intuitive sense because:

- **Skill Development**: More experienced employees have developed valuable skills over time
- **Industry Knowledge**: Years of experience bring deeper understanding of the industry
- **Track Record**: Experienced employees have proven their capabilities
- **Market Value**: The labor market generally values experience with higher compensation

For example, if the Experience coefficient is +1,500, it means that for each additional year of experience, the expected salary increases by approximately $1,500, all else being equal.

---

**2. If Location has a significant effect on salary, what could be the reason?**

Significant location effects on salary can be attributed to:

- **Cost of Living**: Cities like New York or San Francisco have higher costs of living, requiring higher salaries to maintain similar lifestyles
- **Local Labor Market**: Some locations have more competition for talent, driving salaries up
- **Industry Concentration**: Tech hubs (Silicon Valley) or financial centers (NYC) may pay more for specific roles
- **Tax Implications**: State and local taxes vary, affecting take-home pay and gross salary negotiations
- **Economic Development**: More developed regions typically offer higher salaries

---

**3. If Education has a high p-value, what does that suggest about its impact?**

A high p-value (typically > 0.05) for Education suggests that:

- The relationship between education level and salary is **not statistically significant**
- We **cannot confidently conclude** that education level affects salary in this dataset
- The effect might be due to random variation rather than a true relationship

**Possible reasons for non-significance:**

1. **Experience trumps education**: In this dataset, experience might be a stronger predictor, making education's effect less apparent
2. **Role-specific requirements**: Some roles value skills and experience over formal education
3. **Industry variation**: The data might include industries where education matters less
4. **Collinearity**: Education might be correlated with other features (like Experience), diluting its independent effect
5. **Sample characteristics**: The sample might not have enough variation in education levels to detect an effect

**Practical implication**: Organizations should focus on factors that show statistical significance (like experience) when making salary decisions, while being cautious about assumptions regarding education's impact.

---
## 9. Summary and Conclusions

In [ ]:
# Final Summary
print("="*80)
print("FINAL SUMMARY: EMPLOYEE SALARY PREDICTION MODEL")
print("="*80)

print("\n1. MODEL PERFORMANCE:")
print("-"*60)
print(f"   - R-squared (Test): {r2_test:.4f} (explains {r2_test*100:.1f}% of variance)")
print(f"   - RMSE: ${rmse_test:,.2f}")
print(f"   - MAE: ${mae_test:,.2f}")

print("\n2. STATISTICALLY SIGNIFICANT FACTORS:")
print("-"*60)
sig_features = results_df[(results_df['Significant'] == 'Yes') & (results_df['Feature'] != 'const')]
for _, row in sig_features.iterrows():
    impact = "increases" if row['Coefficient'] > 0 else "decreases"
    print(f"   - {row['Feature']}: {impact} salary by ${abs(row['Coefficient']):,.2f}")

print("\n3. KEY INSIGHTS FOR SALARY DECISIONS:")
print("-"*60)
print("   - Experience is typically the strongest predictor of salary")
print("   - Location significantly impacts salary (cost of living adjustment)")
print("   - Education impact varies by industry and role")
print("   - Role type is an important factor in salary determination")

print("\n4. MODEL EQUATION:")
print("-"*60)
print(f"   Salary = ${model.intercept_:,.2f}")
for feature, coef in zip(X.columns, model.coef_):
    if abs(coef) > 1:
        sign = '+' if coef >= 0 else ''
        print(f"            {sign} ${coef:,.2f} x {feature}")

print("\n" + "="*80)
print("END OF TASK 3 SOLUTION")
print("="*80)